# 3. Recuperando resultados de Experimentos

Após a execução de experimentos e salvar seus resultados no MlFlow, é possível utilizar a biblioteca para recuperar tudo que foi salvo.

In [ ]:
import os

import mlflow

# Ao definirmos os valores para essas variáveis de ambiente, 
#   conseguimos acessar os dados guardados no servidor remoto!
os.environ['MLFLOW_TRACKING_URI']= "http://<URL>:8080"
os.environ['MLFLOW_TRACKING_USERNAME']= "<USER>"
os.environ['MLFLOW_TRACKING_PASSWORD']= "<PASSWORD>"

## 3.1 Listagem de experimentos e runs

Inicialmente, precisamos buscar as *runs* que contém esses resultados. O MlFlow provê alguns métodos para listarmos *runs* e experimentos.

In [ ]:
# Podemos obter uma lista dos experimentos fazendo:
mlflow.search_experiments()

In [ ]:
# Também podemos filtrar experimentos
mlflow.search_experiments(filter_string="name = 'Default'")

In [ ]:
# Também podemos filtrar experimentos
mlflow.search_experiments(filter_string="name = 'tutorials/mlflow/iris'")

In [ ]:
# Também podemos filtrar experimentos
mlflow.search_experiments(filter_string="name = '?'")

In [ ]:
# Para listarmos runs, precisamos indicar os grupos de experimentos considerados
mlflow.search_runs(experiment_ids="1")

## 3.2 Recuperando resultados de uma *run*

In [ ]:
# Para recuperamos resultados de uma runa, primeiro
#   vamos precisar do id dela.
run_id = '1b93978f384b4be5b369c2b5edee2612'

# Assim, conseguimos recuperar essa run da seguinte forma:
run = mlflow.get_run(run_id)
run

### 3.2.1 Parâmetros e Métricas

Toda run possui um objeto `RunData` que contém essas informações.

In [ ]:
run.data.params

In [ ]:
run.data.metrics

### 3.2.3 Artefatos

Temos algumas opções para interagir com os artefatos de uma run:

1. Fazer o download do diretório de artefatos para o sistema de arquivos local;
2. Carregar artefatos separadamente;

In [ ]:
# Independente do cenário, é importante recuperar o caminho (URI)
#   onde os artefatos estão salvos no servidor remoto:
artifacts_uri = run.info.artifact_uri
artifacts_uri

In [ ]:
# Podemos carregar os artefatos diretamente
mlflow.artifacts.load_image(f'{artifacts_uri}/confusion_matrix.png')

In [ ]:
# Podemos carregar os artefatos diretamente
mlflow.artifacts.load_image(f'{artifacts_uri}/feature_importance.png')

In [ ]:
# Ou, podemos fazer o download de todos os artefatos:
mlflow.artifacts.download_artifacts(artifacts_uri, 
                                    dst_path='../models/')

### 3.2.4 Datasets

Para obter os datasets de uma run, podemos acessar a propriedade `inputs` do objeto `run`.


In [ ]:
run.inputs.dataset_inputs[0].dataset

In [ ]:
run.inputs.dataset_inputs[1].dataset

### 3.2.5 Modelo

Para obter o modelo de uma run, só precisamos conhecer qual *model flavor* (`pytorch`, `sklearn`, `transformers`, etc) foi utilizado e seu caminho relativo dentro da run.


In [ ]:
# Salvamos o modelo no sistema de arquivos local e carregamos
model = mlflow.sklearn.load_model(f'runs:/{run_id}/extra_trees', 
                                  dst_path='../models/')

In [ ]:
from sklearn import datasets
from sklearn.metrics import classification_report

# Assim, podemos usá-lo diretamente para predições.
dataset = datasets.load_iris()
predictions = model.predict(dataset.data)
print(classification_report(dataset.target,
                            predictions))